In [ ]:
import numpy as np
import pandas as pd
import rasterio

# Load the CSV file as a pandas dataframe
df = pd.read_csv('....path..../CSV.csv')

# Define the dimensions and resolution of the output raster
height = 493
width = 393
resolution = 30  # in meters

# Define the bounds of the output raster in UTM zone 46 projection
xmin = df['longitude'].min()
xmax = df['longitude'].max()
ymin = df['latitude'].min()
ymax = df['latitude'].max()

# Create an empty 2D array to hold the raster data
raster_data = np.zeros((height, width))

# Compute the size of each pixel in the x and y directions
pixel_size_x = (xmax - xmin) / (width - 1)
pixel_size_y = (ymax - ymin) / (height - 1)

# Compute the row and column indices for each point in the dataframe
col_indices = ((df['longitude'] - xmin) / pixel_size_x).astype(int)
row_indices = ((ymax - df['latitude']) / pixel_size_y).astype(int)

# Assign the data values to the corresponding pixels in the raster
raster_data[row_indices, col_indices] = df['data']

# Define the raster's metadata
meta = {
    'driver': 'GTiff',
    'dtype': raster_data.dtype,
    'nodata': 0,
    'width': width,
    'height': height,
    'count': 1,
    'crs': 'EPSG:32646',  # UTM zone 46 projection
    'transform': rasterio.transform.from_bounds(xmin, ymin, xmax, ymax, width, height)
}

# Write the raster to a GeoTIFF file
with rasterio.open('output_Raster.tif', 'w', **meta) as dst:
    dst.write(raster_data, 1)